# APS360 Group Project

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from train import train_net, train_auto_encoder, init_device, test_net
from graph import plot_training_curve, generate_confusion_matrix, visualize_output, visualize_autoencoder_output

In [2]:
init_device()

Cuda Available: False


## Baseline Model

In [3]:
class BaselineModel(nn.Module):
    def __init__(self):
        super(BaselineModel, self).__init__() 
        self.conv1 = nn.Conv2d(1, 5, 3) 
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(5, 10, 3)
        self.conv3 = nn.Conv2d(10, 20, 3)
        self.fc = nn.Linear(26*26*20, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 26*26*20)

        x = self.fc(x)

        return x


In [4]:
train_net(BaselineModel, "baseline_model", learning_rate=0.01, patience=3)

Done Loading Data
Starting Training
Epoch 1: Train err: 0.6415, Train loss: 1.7101116281874635 | Validation err: 1.0, Validation loss: 10.43777847290039


KeyboardInterrupt: 

## Primary Model

In [ ]:
class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 2, 1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, 3, 2, 1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, 3, 2, 1)
        self.bn3 = nn.BatchNorm2d(128)

    def forward(self, x):
        x = self.bn1(F.relu(self.conv1(x)))    # 32 x 112 x 112
        x = self.bn2(F.relu(self.conv2(x)))    # 64 x 56 x 56
        x = self.bn3(F.relu(self.conv3(x)))    # 128 x 28 x 28

        return x

class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()

        self.conv4 = nn.ConvTranspose2d(128, 64, 3, 2, 1, 1)
        self.bn4 = nn.BatchNorm2d(64)
        self.conv5 = nn.ConvTranspose2d(64, 32, 3, 2, 1, 1)
        self.bn5 = nn.BatchNorm2d(32)
        self.conv6 = nn.ConvTranspose2d(32, 1, 3, 2, 1, 1)

    def forward(self, x):

        x = self.bn4(F.relu(self.conv4(x)))
        x = self.bn5(F.relu(self.conv5(x)) )
        x = self.conv6(x)
        
        return x

class AutoEncoder(nn.Module):
    def __init__(self):
        super(AutoEncoder, self).__init__()
        self.encoder = Encoder()
        self.decoder = Decoder()

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)

        return x

class PrimaryModel(nn.Module):
    def __init__(self, encoder, layer_size):
        super(PrimaryModel, self).__init__()
        self.encoder = encoder
        self.encoder.requires_grad_(False)
        self.layer_size = layer_size
        
        self.conv1 = nn.Conv2d(1, layer_size, 3, 2, 1) 
        self.bn1 = nn.BatchNorm2d(layer_size)
        self.conv2 = nn.Conv2d(layer_size, layer_size*2, 3, 2, 1)
        self.bn2 = nn.BatchNorm2d(layer_size*2)
        self.conv3 = nn.Conv2d(layer_size*2, layer_size*4, 3, 2, 1)
        self.bn3 = nn.BatchNorm2d(layer_size*4)

        self.conv4 = nn.Conv2d(128, layer_size*4, 1)
        self.bn4 = nn.BatchNorm2d(layer_size*4)
        
        self.conv5 = nn.Conv2d(layer_size*8,layer_size*2,1)
        self.bn5 = nn.BatchNorm2d(layer_size*2)
        
        self.fc1 = nn.Linear(layer_size*2*28*28, 128)
        self.bn6 = nn.BatchNorm1d(128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        
        x1 = self.bn1(F.relu(self.conv1(x)))
        x1 = self.bn2(F.relu(self.conv2(x1)))
        x1 = self.bn3(F.relu(self.conv3(x1)))
        
        x2 = self.encoder(x)
        x2 = self.bn4(F.relu(self.conv4(x2)))
        
        x = torch.cat((x1,x2), dim=1)
        x = self.bn5(F.relu(self.conv5(x)))
                
        x = x.view(-1, self.layer_size*2*28*28)
        x = self.bn6(F.relu(self.fc1(x)))
        x = F.softmax(self.fc2(x), dim=1)

        return x

In [ ]:
train_auto_encoder(AutoEncoder, "autoencoder", learning_rate=0.002, batch_size=16, patience=10)

In [ ]:
for num_layers in range(1, 9):
    auto_encoder = AutoEncoder()
    auto_encoder.load_state_dict(torch.load("autoencoder\\best_model"))
    train_net(PrimaryModel, "primary_model" + str(num_layers), model_params=[auto_encoder.encoder, num_layers], learning_rate=0.0001, batch_size=32, patience=5)

# Visualization

In [5]:
# Testing
print("Baseline Model: ", end="")
test_net(BaselineModel, "baseline_model\\best_model")
print("Primary Model: ", end="")
test_net(PrimaryModel, "primary_model\\best_model", model_params=[Encoder()])

Baseline Model: Done Loading Data
Test error: 1.0, Test loss: 9.777180314064026
Primary Model: 

NameError: name 'PrimaryModel' is not defined

In [ ]:
# Graphing
plot_training_curve("baseline_model")
plot_training_curve("primary_model")

In [ ]:
# Visualizing
visualize_output(5 ,"baseline_model\\best_model", BaselineModel)
visualize_output(5, "primary_model\\best_model", PrimaryModel, model_params=[Encoder()])

In [ ]:
visualize_autoencoder_output(5, "autoencoder\\best_model", AutoEncoder)


In [ ]:
# Confusion Matrix
generate_confusion_matrix(BaselineModel, "baseline_model\\best_model")
generate_confusion_matrix(PrimaryModel, "primary_model\\best_model")